In [13]:
#automatic differentiation
%matplotlib inline
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True) #w、b需要優化的參數
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

print('Gradient function for z =',z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

print("w before grad: ", w)
print("b before grad: ", b)
loss.backward()
print("w after grad: ", w.grad)
print("b after grad: ", b.grad)

Gradient function for z = <AddBackward0 object at 0x12955af50>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x12955af50>
w before grad:  tensor([[-0.3666, -0.8199,  0.9374],
        [ 0.9585, -0.7290, -0.7043],
        [ 1.4277, -0.6626,  0.1160],
        [ 0.0232,  0.1422, -1.1191],
        [-0.7853, -1.0869, -0.9375]], requires_grad=True)
b before grad:  tensor([ 2.8972,  0.3593, -1.6712], requires_grad=True)
w after grad:  tensor([[0.3282, 0.0192, 0.0110],
        [0.3282, 0.0192, 0.0110],
        [0.3282, 0.0192, 0.0110],
        [0.3282, 0.0192, 0.0110],
        [0.3282, 0.0192, 0.0110]])
b after grad:  tensor([0.3282, 0.0192, 0.0110])


In [14]:
#disable grad: To mark some parameters in your neural network as frozen parameters
#or to speed up computations when you are only doing a forward pass
#way 1
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

#way 2
z_det = z.detach()
print(z_det.requires_grad)

True
False
False


In [15]:

#Optimizing the model parameters
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [16]:
#setting hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 5
#Initialize the loss function
loss_fn = nn.CrossEntropyLoss

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [21]:
#full implementation
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad() #back propagation
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {100*correct:>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [20]:
print(training_data)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [22]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    
print('Done!')

Epoch 1
-----------------------------------
loss: 2.270910 [    0/60000]
loss: 2.265785 [ 6400/60000]
loss: 2.234964 [12800/60000]
loss: 2.232629 [19200/60000]
loss: 2.228119 [25600/60000]
loss: 2.202850 [32000/60000]
loss: 2.204820 [38400/60000]
loss: 2.176862 [44800/60000]
loss: 2.187099 [51200/60000]
loss: 2.140167 [57600/60000]
Test Error: 
 Accuracy: 53.9%, Avg loss: 0.033659 

Epoch 2
-----------------------------------
loss: 2.166394 [    0/60000]
loss: 2.157312 [ 6400/60000]
loss: 2.074025 [12800/60000]
loss: 2.078578 [19200/60000]
loss: 2.075523 [25600/60000]
loss: 2.022136 [32000/60000]
loss: 2.037387 [38400/60000]
loss: 1.977689 [44800/60000]
loss: 2.002909 [51200/60000]
loss: 1.900041 [57600/60000]
Test Error: 
 Accuracy: 54.6%, Avg loss: 0.030165 

Epoch 3
-----------------------------------
loss: 1.975326 [    0/60000]
loss: 1.957195 [ 6400/60000]
loss: 1.793568 [12800/60000]
loss: 1.810485 [19200/60000]
loss: 1.835393 [25600/60000]
loss: 1.736969 [32000/60000]
loss: 1.77

In [12]:
#saving models

torch.save(model.state_dict(), "data/model.pth")
print("Saved PyTorch State to model.pth")

Saved PyTorch State to model.pth
